In [3]:
from functions.adversarial import *
from functions.classification import *
import cv2
import numpy as np
from utils import *
import matplotlib.pyplot as plt
from functions.imagenet_classes import classes

## Import images and classify

In [4]:
adv, orig, pert = adversarial_examples("data/goldfish.jpg",method = "LLCI",eps=100, num_iter=125,show=False)

In [ ]:
# Set image path
im_path = 'results/Adv_DIP/goldfish/Std_complex_16-64'
original_class = 1  #1 is for goldfish
P, R = classification(adv, model_name = 'resnet18', sort = True, show=False)
final_classes = R[0,0:5]
iterations = 9900

Confidence = np.ones([round(iterations/100+1),6])
Ranks_matrix = np.ones([round(iterations/100+1),5])

for i in range(round(iterations/100+1)):
    orig = cv2.imread('{}/it_{}.png'.format(im_path,i*100))[..., ::-1]
    orig = cv2.resize(orig, (256, 256))
    img = orig.copy().astype(np.float32)
    Probs, Ranks = classification(img, model_name = 'resnet18', sort = False, show = False)
    Probs_np = torch_to_np(Probs)
    Confidence[i,0] = Probs_np[original_class]
    P , Ranking = Probs.sort(descending=True)
    Ranking_np = torch_to_np(Ranking)
    for j in range(5):
        Confidence[i,j+1] = Probs_np[final_classes[j]]
        Ranks_matrix[i,j] = Ranking_np[j]
        
np.savetxt('{}/Confidences.txt'.format(im_path), Confidence)
np.savetxt('{}/Ranks.txt'.format(im_path), Ranks_matrix)